## SVM Classification

In [1]:
from numpy import linalg
from menpo import io as mio
from menpo.visualize import print_dynamic
from sklearn.utils.fixes import bincount
import itertools
import warnings
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from dAAMs.lda import lda, predict, chunk, n_fold_generate
from menpo.feature import igo, hog, no_op, double_igo as digo, dsift, fast_dsift, hellinger_vector_128_dsift
from sklearn import svm
import numpy as np

%matplotlib inline

ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.
Using Theano backend.


### verification generation

In [2]:
import itertools

In [35]:
images = mio.import_images("/homes/yz4009/wd/databases/ear/EarVerification/VGGEAR/bound/")

In [36]:
images[0].path.name

'000-0593.jpg'

In [37]:
data = []
for img in mio.import_images("/homes/yz4009/wd/databases/ear/EarVerification/VGGEAR/bound/"):
    label = int(img.path.stem.split("-")[0])
    name = img.path.name
    
    data.append((label,name))

In [38]:
def pair_generation(data, n_folds=3):
    np.random.seed(10)
    n_data = len(data)
    n_id = len(np.unique([d[0] for d in data]))
    fold_limit = n_data / n_folds
    grouped_data = itertools.groupby(data, lambda x:x[0])
    
    positives = []
    negatives = []
    
    folds = []
    
    def if_exist(item, lists):
        return np.array([item[0] == l and (item[1] == d).all() for l,d in lists]).any()
    
    def fold_process(pos, neg):
        while len(neg) < len(pos) / 2:
            r_pos = pos[np.random.randint(0,len(pos))]
            
            try:
                isexist = r_pos in neg
            except:
                isexist = if_exist(r_pos, neg)
            
            
            if not isexist:
#             if not r_pos in neg:
                neg.append(r_pos)

        pair_negs = []
        for r_neg in neg:
            found = False
            while not found:
                r_pos = pos[np.random.randint(0,len(pos))]
                if not r_pos[0] == r_neg[0]:
                    pair_negs.append(r_pos)
                    found = True

        return list(zip(pos[::2],pos[1::2])), list(zip(neg,pair_negs))
    
    for identity,g_it in grouped_data:
        
        p_list = []
        for d in g_it:
            p_list.append(d)
        
        if len(p_list) % 2 > 0:
            negatives.append(p_list.pop())
            
        positives += p_list
        
        if len(positives) + len(negatives) > fold_limit:
            
            folds.append(fold_process(positives, negatives))
            
            positives = []
            negatives = []
    
    folds.append(fold_process(positives, negatives))
    
    return folds

In [9]:
# data = mio.import_pickle('/homes/yz4009/wd/PickleModel/EarRecognition/LDA-VGG-Data-dsift.pkl')
# data = mio.import_pickle('/homes/yz4009/wd/PickleModel/EarRecognition/LDA-VGG-Data-dsift.pkl', encoding='latin1')
# data = mio.import_pickle('/homes/yz4009/wd/PickleModel/EarRecognition/LDA-VGG-Data.pkl', encoding='latin1')
# data = mio.import_pickle('/homes/yz4009/wd/PickleModel/EarRecognition/VGGEAR-bound-PEP.pkl', encoding='latin1')
data = mio.import_pickle('/homes/yz4009/wd/PickleModel/EarRecognition/WPUTEDB-.pkl', encoding='latin1')

In [39]:
%%time

folds = pair_generation(data, n_folds=5)

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 14 ms


In [41]:
print(len(folds[0][0]))

199


In [42]:
new_folds = []
for p,n in folds:
    np.random.shuffle(p)
    np.random.shuffle(n)
    new_folds.append([p[:185],n[:185]])

In [44]:
print(len(new_folds[0][0]))

185


In [11]:
mio.export_pickle(folds,'/homes/yz4009/wd/PickleModel/EarRecognition/WPUTEDB-5folds-PEP.pkl', overwrite=True)

In [6]:
mio.export_pickle(folds,'/homes/yz4009/wd/PickleModel/EarRecognition/VGGEAR-5folds-PEP.pkl', overwrite=True)

In [45]:
mio.export_pickle(new_folds,"/homes/yz4009/wd/databases/ear/EarVerification/VGGEAR/protocol.pkl")

In [28]:
folds = mio.import_pickle("/homes/yz4009/wd/databases/ear/EarVerification/WPUTEDB/protocol-yx.pkl")

In [29]:
new_folds = []
for pos,neg in folds:
    one_fold = []
    for (_,i1),(_,i2) in pos[:185]:
        one_fold.append([[i1,i2],1])
    for (_,i1),(_,i2) in neg[:185]:
        one_fold.append([[i1,i2],0])
    new_folds.append(one_fold)

In [33]:
mio.export_pickle(new_folds, "/homes/yz4009/wd/databases/ear/EarVerification/WPUTEDB/protocol.pkl", overwrite=True)

In [32]:
len(new_folds[0])

370